# Backtrack Testbed Notebook

Short notebook to test various interpolation strategies. 

In [3]:
# Standard library
import sys

# Data manipulation and analysis
import xarray as xr

In [4]:
sys.path.insert(1, '../sealsml')

In [5]:
from backtrack import polar